# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%%bash
./refreshdata local

---
## Predict cases for all countries (requires pre-training)

From `./work` please run

```
python covidvu/predict.py all
```

before running these cells

In [ ]:
from covidvu.pipeline.vujson import parseCSSE
from covidvu.predict import load
from covidvu.predict import getSavedShortCountryNames
from covidvu.visualize import plotPrediction
import pandas as pd
import numpy as np

In [ ]:
from ipywidgets import interact
from ipywidgets import widgets

In [ ]:
confirmedCasesAll = parseCSSE('confirmed')['casesGlobal']

In [ ]:
nTrainedCountries = len(getSavedShortCountryNames())

In [ ]:
multiCountry = widgets.SelectMultiple(
    options=np.arange(nTrainedCountries),
    value=[0],
    description='Country Index',
    disabled=False
)
log = widgets.Checkbox(value=False, description='Log scale')

In [ ]:
def plotAllCountries(countryToLoadIndex, log):    
    countryToLoadIndex = int(countryToLoadIndex[0])
    meanPredictionTS, percentilesTS, countryName =  load(countryToLoadIndex)
    data = confirmedCasesAll[countryName]
    data.index = pd.to_datetime(data.index)
    fig, ax = plotPrediction(data, meanPredictionTS, percentilesTS, countryName, log)

In [ ]:
interact(plotAllCountries, countryToLoadIndex=multiCountry, log=log);

---
## Predict cases for a single country

In [ ]:
COUNTRY_NAME = 'China'

In [ ]:
from covidvu.utils import autoReloadCode; autoReloadCode()
from covidvu.predict import predictLogisticGrowth
import matplotlib.pyplot as plt
import pymc3 as pm
import numpy as np

Recommended values should be greater than, or equal to:

- `nTune = 200`
- `nSamples = 500`
- `nChains = 2`
- `nBurn = 100`

(Running on Jupyter notebooks is for demonstration purposes only, and will be much slower than running `python covidvu/predict.py <i>`)

In [ ]:
nTune = 200
nSamples = 500
nChains = 2
nBurn = 100

This may take some time...

In [ ]:
predictions = predictLogisticGrowth(
    countryName=COUNTRY_NAME,
    nTune=nTune,
    nSamples=nSamples,
    nChains=nChains,
    nBurn=nBurn,
)

In [ ]:
pm.plots.traceplot(predictions['trace'][nBurn:]);

In [ ]:
fig, ax = vis.plot()
predictions['predictionsMeanTS'].plot(ax=ax, linestyle='-.', color='black', label='Mean Prediction')
predictions['countryTSClean'].plot(ax=ax, marker='o', color='green', label='Data')
ax.fill_between(predictions['predictionsMeanTS'].index,
                predictions['predictionsPercentilesTS'][0][0],
                predictions['predictionsPercentilesTS'][0][1],
                color='red',
                alpha=0.1, 
                label = r"95% CI");

ax.fill_between(predictions['predictionsMeanTS'].index,
                predictions['predictionsPercentilesTS'][1][0],
                predictions['predictionsPercentilesTS'][1][1],
                color='red',
                alpha=0.5, 
                label = r"50% CI");

    
ax.set_ylabel('Total confirmed cases')
ax.set_title(COUNTRY_NAME)
ax.legend(loc='upper left')
plt.savefig(f'./figures/{COUNTRY_NAME}_prediction.png', bbox_inches='tight')

In [ ]:
fig, ax = vis.plot()
np.log10(predictions['predictionsMeanTS']+1).plot(ax=ax, linestyle='-.', color='black', label='Mean Prediction')
np.log10(predictions['countryTSClean']+1).plot(ax=ax, marker='o', color='green', label='Data clean')
ax.fill_between(predictions['predictionsMeanTS'].index,
                np.log10(predictions['predictionsPercentilesTS'][0][0]+1),
                np.log10(predictions['predictionsPercentilesTS'][0][1]+1),
                color='red',
                alpha=0.1, 
                label = r"95% CI");

ax.fill_between(predictions['predictionsMeanTS'].index,
                np.log10(predictions['predictionsPercentilesTS'][1][0]+1),
                np.log10(predictions['predictionsPercentilesTS'][1][1]+1),
                color='red',
                alpha=0.5, 
                label = r"50% CI");
    
ax.set_ylabel('Log 10 Total confirmed cases')
ax.set_title(COUNTRY_NAME)
ax.legend(loc='upper left')
plt.savefig(f'./figures/{COUNTRY_NAME.strip()}_log_prediction.png', bbox_inches='tight')

---
&#169; the COVIDvu Contributors.  All rights reserved.